In [1]:
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import VGG16

In [2]:
import os
import tensorflow as tf

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
data = pd.read_csv('emnist-byclass-mapping.txt', sep=" ", header=None)
data.columns = ['Target','ASCII']

col=[]
for i in data.ASCII.tolist():
    col.append(chr(i))

data['ASCII_Target'] = pd.Series(col)

In [4]:
train_df = pd.read_csv('emnist-byclass-train.csv')
train_df.rename(columns={'35':'Target'}, inplace=True)
train_y = np.array(train_df.pop('Target'))
train_x = train_df

train_x = train_x / 255.0
train_x = train_x.values.reshape([-1, 28, 28, 1])
train_y = keras.utils.to_categorical(train_y)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [5]:
test_df = pd.read_csv('emnist-byclass-test.csv')
test_df.rename(columns={'18':'Target'}, inplace=True)

# test_df=pd.merge(test_df, data, how='left', on='Target')
# test_df=test_df.drop(['Target','ASCII'],axis=1)
# test_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

test_y = np.array(test_df.pop('Target'))
test_x = test_df

test_x = test_x / 255.0
test_x = test_x.values.reshape([-1, 28, 28, 1])
test_y = keras.utils.to_categorical(test_y)


In [6]:
hand_df = pd.read_csv('our_handmade_dataset.csv')
hand_df.rename(columns={'0':'Target'}, inplace=True)

# hand_df=pd.merge(hand_df, data, how='left', on='Target')
# hand_df=hand_df.drop(['Target','ASCII'],axis=1)
# hand_df.rename(columns={'ASCII_Target':'Target'}, inplace=True)

hand_y = np.array(hand_df.pop('Target'))
hand_x = hand_df

hand_x = hand_x / 255.0
hand_x = hand_x.values.reshape([-1, 28, 28, 1])
hand_y = keras.utils.to_categorical(hand_y)

In [7]:
inputs = Input(shape = (28,28,1))
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(inputs)
x = Conv2D (filters =64, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 2nd Conv Block
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =128, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 3rd Conv block  
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x) 
x = Conv2D (filters =256, kernel_size =3, padding ='same', activation='relu')(x) 
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 4th Conv block
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# 5th Conv block
# x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
# x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
# x = Conv2D (filters =512, kernel_size =3, padding ='same', activation='relu')(x)
# x = MaxPool2D(pool_size =2, strides =2, padding ='same')(x)
# Fully connected layers  
x = Flatten()(x)
x = Dense(units = 4096, activation ='relu')(x)
x = Dense(units = 4096, activation ='relu')(x)
output = Dense(units = 62, activation ='softmax')(x)
# creating the model

vgg_emnist_model = Model (inputs=inputs, outputs =output)
vgg_emnist_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 128)       147584    
                                                             

In [10]:
vgg_emnist_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.01), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_vgg_emnist_model.h5", save_best_only=True)
  
vgg_emnist_model.fit(train_x, train_y, batch_size = 512, epochs=100, validation_data=(val_x, val_y), callbacks=[early_stopping_cb, model_checkpoint_cb])

vgg_emnist_model.evaluate(test_x, test_y)

Epoch 1/100
1091/1091 [==============================] - 88s 79ms/step - loss: 3.6860 - accuracy: 0.0540 - val_loss: 3.6842 - val_accuracy: 0.0556
Epoch 2/100
1091/1091 [==============================] - 88s 81ms/step - loss: 3.6856 - accuracy: 0.0541 - val_loss: 3.6843 - val_accuracy: 0.0556
Epoch 3/100
1091/1091 [==============================] - 86s 79ms/step - loss: 3.6855 - accuracy: 0.0540 - val_loss: 3.6844 - val_accuracy: 0.0556
Epoch 4/100
1091/1091 [==============================] - 86s 79ms/step - loss: 3.6854 - accuracy: 0.0543 - val_loss: 3.6840 - val_accuracy: 0.0556
Epoch 5/100
1091/1091 [==============================] - 86s 79ms/step - loss: 3.6854 - accuracy: 0.0542 - val_loss: 3.6837 - val_accuracy: 0.0556
Epoch 6/100
1091/1091 [==============================] - 82s 75ms/step - loss: 3.6853 - accuracy: 0.0542 - val_loss: 3.6838 - val_accuracy: 0.0556
Epoch 7/100
1091/1091 [==============================] - 82s 75ms/step - loss: 3.6851 - accuracy: 0.0546 - val_loss: 3

[3.6790006160736084, 0.05441790819168091]

In [51]:
my_lenet_5_selu_model.evaluate(hand_x, hand_y)